In [91]:
from qiskit.quantum_info import SparsePauliOp
a=-0.1#float(input("Enter the value of alpha:"))
j=1#float(input("Enter the value of J:"))
H_z=SparsePauliOp.from_list([('IIZZ',j),('IZZI',j),('ZZII',j),('ZIIZ',j),('IZIZ',a*j),('ZIZI',a*j)])
H_x=SparsePauliOp.from_list([('IIXX',j),('IXXI',j),('XXII',j),('XIIX',j),('IXIX',a*j),('XIXI',a*j)])
H_y=SparsePauliOp.from_list([('IIYY',j),('IYYI',j),('YYII',j),('YIIY',j),('IYIY',a*j),('YIYI',a*j)])
hamiltonian=H_x+H_y+H_z

In [92]:
from qiskit.circuit.library import QAOAAnsatz,EfficientSU2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService, Estimator

service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

In [93]:
#QAOA ansatz parameters
#optimised parameters
theta1= [ 1.38684792,  4.71815379,  5.1955337 ,  2.81505774,  5.54081294,
        3.65764277,  2.94022964, -5.66795214, -1.19603113,  3.71736527]

#unoptimised/random parameters
theta2=[ 3.18811673e-01,  3.71574924e+00,  4.91507708e+00,  4.10014814e+00,
        2.35620626e-03, -4.40751099e-01, -5.14034744e+00, -3.85325702e+00,
        1.76417911e+00, -4.13018813e+00]

#semioptimised parameters
theta3=[ 5.41221168e+00,  3.09268073e+00,  1.69005444e+00,  2.92075991e+00,
        1.05231218e+00,  4.47644271e+03,  2.09285202e+04, -4.35150366e+02,
       -1.07147755e+03, -3.80464401e+03]

In [94]:
#EfficientSU2 ansatz parameters
#optimised parameters
theta4=[-2.8266564 , -3.13407933,  2.61085316, -0.11746563,  1.35511505,
        0.550615  ,  1.18968703,  2.9758423 ,  2.31668326,  0.10281567,
        1.81412336,  2.92500781, -1.70427766,  2.89010607,  1.70025257,
        1.83630034, -0.40907205,  1.43997164,  0.57360503,  1.09171809,
       -0.81921012,  2.1208924 ,  1.39237549,  1.25764986,  0.20589422,
        2.44965689,  1.54972208, -0.17529336,  1.2239305 , -2.01536281,
        3.14079523,  0.59181308, -1.7411251 , -1.86236206,  1.30262758,
        0.45495128,  0.97569567, -1.76797766, -1.27535669, -1.50693083,
       -0.07287223, -2.85458363,  0.38738485, -3.0172031 , -2.31894707,
        1.76531595,  2.52018897,  3.04363669]
#unoptimised/random parameters
theta5=[-0.42460481,  0.10015925,  0.02516064,  1.46677671,  1.47426429,
       -2.90862465,  2.91307923,  0.15382639, -1.07329674, -0.41394104,
       -2.26064863, -1.02903633, -2.4770042 ,  2.67599274,  2.66254868,
        3.08716381, -0.01955244, -0.26958608, -1.70647119,  2.52656215,
        0.38561245,  1.99492168,  2.94066229, -3.03871018, -2.67268977,
       -1.37413109, -2.01314074, -1.71096286, -0.09980921,  0.33155178,
        2.08237522,  0.19488875,  1.25306982,  2.37799985, -1.41581474,
        2.81652794,  0.56761778,  2.16317226, -2.26895932, -0.64181942,
       -2.09981362,  3.02316214, -1.58258047, -0.92131062,  2.13215558,
        0.77703431, -0.72884201,  2.85670204]
#semioptimised parameters
theta6=[ 1.4069061 , -1.85519084, -0.39681994,  0.64621524,  2.91452562,
        0.8945616 , -2.61258861,  1.4843819 , -0.23524628, -2.58486456,
       -0.17265946, -1.76729787,  0.49671256,  2.70113273, -2.62052214,
       -1.56512988,  0.54060762,  0.71061121, -0.6087021 ,  0.76211731,
        1.34140104,  2.24806053,  1.9685591 ,  2.99611165,  1.1265912 ,
       -2.02879808, -2.08653089,  1.50117786, -2.98131772,  0.50146195,
       -0.85774548,  0.98849495,  1.36786824, -2.92815174, -0.74974056,
       -0.24197039, -2.85596922,  0.97497039, -2.1029854 , -1.99609237,
        0.56184173, -1.91132681,  2.68022374,  0.33164594, -1.59874629,
       -0.8437977 , -2.82307569,  1.59569365]

In [108]:
params=theta1
psi= QAOAAnsatz(hamiltonian,reps=5)#EfficientSU2(4,reps=5)#use this to run VQE
pm = generate_preset_pass_manager(backend=backend, optimization_level=2)
isa_psi = pm.run(psi)
isa_observables = hamiltonian.apply_layout(isa_psi.layout)

estimator = Estimator(backend=backend)
job=estimator.run(circuits=isa_psi, observables=isa_observables, parameter_values=params)

In [ ]:
pub_result = job.result()
pub_result

In [107]:
#to run it on simulator
from qiskit.circuit.library import QAOAAnsatz
from qiskit.primitives import Estimator
psi= QAOAAnsatz(hamiltonian,reps=5)
#psi=EfficientSU2(4,reps=5)  #use this to run VQE
exact_estimator = Estimator()
job = exact_estimator.run(psi, hamiltonian, theta4,shots=30)#change theta parameters accordingly
exact_value = job.result()
exact_value


EstimatorResult(values=array([-8.18333353]), metadata=[{'variance': 0.024034284523125393, 'shots': 30}])